In [55]:
from ctypes import *
import random
from PIL import Image, ImageOps
import numpy as np
import os
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, accuracy_score

my_dll = cdll.LoadLibrary(
        "C:/Users/ariel/OneDrive/Bureau/ESGI/Bachelor/Projet Annuel/Library C++/Library_MLP/cmake-build-debug/Library_MLP.dll")

In [56]:
def create_mlp_model(struct_model):
    struct_model_int = [int(i) for i in struct_model]
    struct_model_type = len(struct_model_int) * c_int

    my_dll.create_mlp_model.argtypes = [struct_model_type,
                                        c_int]
    my_dll.create_mlp_model.restype = POINTER(c_void_p)

    return my_dll.create_mlp_model(struct_model_type(*struct_model_int), len(struct_model_int))

In [57]:
def predict_mlp(model, input, len_input, is_classification):
    my_dll.predict_mlp_model.argtypes = [c_void_p, POINTER(c_float), c_int32,
                                         c_int32]
    my_dll.predict_mlp_model.restype = POINTER(c_float)
    ng = c_float * len(input)
    predictions = my_dll.predict_mlp_model(model, ng(*input), len_input, is_classification)
    return predictions[1]

In [58]:
def train_mlp(model_to_train, X_to_train, y, learning_rate, nb_iter, is_classification):
    my_dll.alternate_train_MLP.argtypes = [c_void_p, POINTER(c_float),
                                           POINTER(c_float), c_float, c_int32]
    my_dll.alternate_train_MLP.restype = None

    for i in tqdm(range(0, nb_iter)):
        k = random.choice(list(range(0, len(X_to_train))))
        arr_types_X = c_float * len(X_to_train[k])
        arr_types_Y = c_float * len(y[k])
        my_dll.alternate_train_MLP(model_to_train, arr_types_X(*X_to_train[k]), arr_types_Y(*y[k]), learning_rate,
                                   is_classification)

In [59]:
def destroy_model(model_to_destroy):
    my_dll.destroy_mlp_model.argtypes = [c_void_p]
    my_dll.destroy_mlp_model.restype = None

    my_dll.destroy_mlp_model(model_to_destroy)

In [60]:
# Linear Simple
X = np.array([
      [1, 1],
      [2, 3],
      [3, 3]
])
Y = np.array([
      1,
      -1,
      -1
])
Y = [[i] for i in Y]

npl = [2, 2, 1]

model = create_mlp_model(npl)

print("Before Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

print("\nTraining...")
train_mlp(model, X, Y, 0.1, 10000, 1)

print("After Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

destroy_model(model)

 12%|█████████▏                                                                | 1241/10000 [00:00<00:00, 12360.12it/s]

Before Training :
True : [1] Pred : 0.6578037738800049
True : [-1] Pred : 0.6627044677734375
True : [-1] Pred : 0.6894180774688721

Training...


100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 13712.38it/s]

After Training :
True : [1] Pred : 0.9852288961410522
True : [-1] Pred : -0.9890769720077515
True : [-1] Pred : -0.989264965057373


In [62]:
# Linear Multiple
X = np.concatenate([np.random.random((50,2)) * 0.9 + np.array([1, 1]), np.random.random((50,2)) * 0.9 + np.array([2, 2])])
Y = np.concatenate([np.ones((50, 1)), np.ones((50, 1)) * -1.0])

Y = [[i] for i in Y]

npl = [2, 2, 1]

model = create_mlp_model(npl)

print("Before Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

print("\nTraining...")
train_mlp(model, X, Y, 0.01, 10000, 1)

print("After Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

destroy_model(model)

Before Training :
True : [array([1.])] Pred : 0.6612188816070557
True : [array([1.])] Pred : 0.6818639039993286
True : [array([1.])] Pred : 0.6724065542221069
True : [array([1.])] Pred : 0.6607739925384521
True : [array([1.])] Pred : 0.6725496649742126
True : [array([1.])] Pred : 0.6791455745697021
True : [array([1.])] Pred : 0.6772385835647583
True : [array([1.])] Pred : 0.6505860090255737
True : [array([1.])] Pred : 0.6618598699569702
True : [array([1.])] Pred : 0.6802562475204468
True : [array([1.])] Pred : 0.6630656123161316
True : [array([1.])] Pred : 0.6660841703414917
True : [array([1.])] Pred : 0.6667900681495667
True : [array([1.])] Pred : 0.6659759879112244
True : [array([1.])] Pred : 0.6543796062469482
True : [array([1.])] Pred : 0.6821314692497253
True : [array([1.])] Pred : 0.666630208492279
True : [array([1.])] Pred : 0.6490058302879333
True : [array([1.])] Pred : 0.6657251715660095
True : [array([1.])] Pred : 0.6728650331497192
True : [array([1.])] Pred : 0.6829218864440

  0%|                                                                                        | 0/10000 [00:00<?, ?it/s]


True : [array([-1.])] Pred : 0.6870902180671692
True : [array([-1.])] Pred : 0.6716049313545227
True : [array([-1.])] Pred : 0.6912031769752502
True : [array([-1.])] Pred : 0.6769068241119385
True : [array([-1.])] Pred : 0.6798335909843445
True : [array([-1.])] Pred : 0.6840022206306458

Training...


100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 17564.04it/s]


After Training :
True : [array([1.])] Pred : 0.988115668296814
True : [array([1.])] Pred : 0.891446590423584
True : [array([1.])] Pred : 0.9385326504707336
True : [array([1.])] Pred : 0.9958847761154175
True : [array([1.])] Pred : 0.9968345761299133
True : [array([1.])] Pred : 0.7370153665542603
True : [array([1.])] Pred : 0.9922634363174438
True : [array([1.])] Pred : 0.991682767868042
True : [array([1.])] Pred : 0.9977841973304749
True : [array([1.])] Pred : 0.9683206677436829
True : [array([1.])] Pred : 0.9874264597892761
True : [array([1.])] Pred : 0.9603743553161621
True : [array([1.])] Pred : 0.9940261840820312
True : [array([1.])] Pred : 0.9922917485237122
True : [array([1.])] Pred : 0.9971880316734314
True : [array([1.])] Pred : 0.8017272353172302
True : [array([1.])] Pred : 0.9846189022064209
True : [array([1.])] Pred : 0.9859250783920288
True : [array([1.])] Pred : 0.9418208599090576
True : [array([1.])] Pred : 0.9957359433174133
True : [array([1.])] Pred : 0.9947001338005066

In [63]:
# XOR
X = np.array([[1, 0], [0, 1], [0, 0], [1, 1]])
Y = np.array([1, 1, -1, -1])
Y = [[i] for i in Y]

npl = [2, 2, 1]

model = create_mlp_model(npl)

print("Before Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

print("\nTraining...")
train_mlp(model, X, Y, 0.1, 10000, 1)

print("After Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

destroy_model(model)

  0%|                                                                                        | 0/10000 [00:00<?, ?it/s]

Before Training :
True : [1] Pred : 0.6695356965065002
True : [1] Pred : 0.5567963123321533
True : [-1] Pred : 0.5449346303939819
True : [-1] Pred : 0.6578037738800049

Training...


100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 20764.77it/s]

After Training :
True : [1] Pred : 0.9367290735244751
True : [1] Pred : 0.938219428062439
True : [-1] Pred : -0.9464904069900513
True : [-1] Pred : -0.9181125164031982


In [64]:
# Cross
X = np.random.random((500, 2)) * 2.0 - 1.0
Y = np.array([1 if abs(p[0]) <= 0.3 or abs(p[1]) <= 0.3 else -1 for p in X])
Y = [[i] for i in Y]

npl = [2, 4, 1]

model = create_mlp_model(npl)

print("Before Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

print("\nTraining...")
train_mlp(model, X, Y, 0.1, 100000, 1)

print("After Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

destroy_model(model)

Before Training :
True : [1] Pred : 0.33303511142730713
True : [-1] Pred : 0.466105580329895
True : [1] Pred : 0.31078973412513733
True : [-1] Pred : 0.31488439440727234
True : [1] Pred : 0.3171393871307373
True : [1] Pred : 0.3237871527671814
True : [-1] Pred : 0.35478314757347107
True : [1] Pred : 0.3458186984062195
True : [-1] Pred : 0.4569995701313019
True : [-1] Pred : 0.49753034114837646
True : [1] Pred : 0.3285461366176605
True : [1] Pred : 0.353771835565567
True : [1] Pred : 0.26304227113723755
True : [1] Pred : 0.256185382604599
True : [1] Pred : 0.29393714666366577
True : [1] Pred : 0.3020016849040985
True : [-1] Pred : 0.32319197058677673
True : [-1] Pred : 0.3288848400115967
True : [1] Pred : 0.2909940481185913
True : [1] Pred : 0.30777624249458313
True : [1] Pred : 0.34993499517440796
True : [1] Pred : 0.29077011346817017
True : [1] Pred : 0.30250248312950134
True : [1] Pred : 0.3209953308105469
True : [1] Pred : 0.3611125946044922
True : [1] Pred : 0.34964287281036377
Tru

True : [1] Pred : 0.2928411662578583
True : [1] Pred : 0.2810096740722656
True : [-1] Pred : 0.3409601151943207
True : [1] Pred : 0.371086061000824
True : [-1] Pred : 0.24194350838661194
True : [1] Pred : 0.285327672958374
True : [-1] Pred : 0.2584437131881714
True : [-1] Pred : 0.33281251788139343
True : [-1] Pred : 0.3165855407714844
True : [-1] Pred : 0.36134204268455505
True : [1] Pred : 0.31928208470344543
True : [-1] Pred : 0.2611379027366638
True : [-1] Pred : 0.42437461018562317
True : [1] Pred : 0.3255515992641449
True : [-1] Pred : 0.3655579388141632
True : [1] Pred : 0.32737889885902405
True : [1] Pred : 0.28788697719573975
True : [1] Pred : 0.40347275137901306
True : [1] Pred : 0.3082481622695923
True : [-1] Pred : 0.30864715576171875
True : [1] Pred : 0.29853421449661255
True : [1] Pred : 0.24734945595264435
True : [-1] Pred : 0.36148500442504883
True : [1] Pred : 0.2614099085330963
True : [-1] Pred : 0.42966535687446594
True : [-1] Pred : 0.37507176399230957
True : [-1] P

100%|███████████████████████████████████████████████████████████████████████| 100000/100000 [00:06<00:00, 14755.35it/s]


After Training :
True : [1] Pred : -0.6246282458305359
True : [-1] Pred : -0.6908859014511108
True : [1] Pred : -0.6628345251083374
True : [-1] Pred : -0.6543295979499817
True : [1] Pred : -0.6687300205230713
True : [1] Pred : -0.6469069123268127
True : [-1] Pred : -0.5938683152198792
True : [1] Pred : 0.9989777207374573
True : [-1] Pred : -0.7021106481552124
True : [-1] Pred : -0.7058953046798706
True : [1] Pred : -0.6330004930496216
True : [1] Pred : -0.6831705570220947
True : [1] Pred : 0.9153622984886169
True : [1] Pred : 0.8349941968917847
True : [1] Pred : 0.9980708360671997
True : [1] Pred : 0.6134769320487976
True : [-1] Pred : -0.654741644859314
True : [-1] Pred : -0.6477572321891785
True : [1] Pred : 0.9999731183052063
True : [1] Pred : 0.9999970197677612
True : [1] Pred : -0.6699105501174927
True : [1] Pred : -0.5579310655593872
True : [1] Pred : -0.6464750170707703
True : [1] Pred : 0.9999489188194275
True : [1] Pred : 0.7014100551605225
True : [1] Pred : -0.663779318332672

In [65]:
# Multi Linear 3 classes
X = np.random.random((500, 2)) * 2.0 - 1.0
Y = np.array([[1, 0, 0] if -p[0] - p[1] - 0.5 > 0 and p[1] < 0 and p[0] - p[1] - 0.5 < 0 else 
              [0, 1, 0] if -p[0] - p[1] - 0.5 < 0 and p[1] > 0 and p[0] - p[1] - 0.5 < 0 else 
              [0, 0, 1] if -p[0] - p[1] - 0.5 < 0 and p[1] < 0 and p[0] - p[1] - 0.5 > 0 else 
              [0, 0, 0]for p in X])

X = X[[not np.all(arr == [0, 0, 0]) for arr in Y]]
Y = Y[[not np.all(arr == [0, 0, 0]) for arr in Y]]

Y = [[i] for i in Y]

npl = [2, 3]

model = create_mlp_model(npl)

print("Before Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

print("\nTraining...")
train_mlp(model, X, Y, 0.1, 100000, 1)

print("After Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

destroy_model(model)

Before Training :
True : [array([0, 1, 0])] Pred : 0.6797896027565002
True : [array([0, 1, 0])] Pred : 0.5902225971221924
True : [array([0, 1, 0])] Pred : 0.5299220681190491
True : [array([0, 1, 0])] Pred : 0.6826459765434265
True : [array([0, 0, 1])] Pred : 0.6365318894386292
True : [array([0, 1, 0])] Pred : 0.5128260254859924
True : [array([0, 1, 0])] Pred : 0.43063685297966003
True : [array([0, 1, 0])] Pred : 0.6800966262817383
True : [array([0, 1, 0])] Pred : 0.6414284706115723
True : [array([0, 0, 1])] Pred : 0.6131302714347839
True : [array([1, 0, 0])] Pred : 0.2927170991897583
True : [array([0, 0, 1])] Pred : 0.6119155287742615
True : [array([0, 1, 0])] Pred : 0.5102279782295227
True : [array([0, 1, 0])] Pred : 0.5331730246543884
True : [array([0, 0, 1])] Pred : 0.6153099536895752
True : [array([0, 1, 0])] Pred : 0.6835718154907227
True : [array([0, 0, 1])] Pred : 0.5930849313735962
True : [array([0, 0, 1])] Pred : 0.6147090792655945
True : [array([1, 0, 0])] Pred : 0.2864777147

True : [array([0, 1, 0])] Pred : 0.6615384817123413
True : [array([0, 0, 1])] Pred : 0.5957841873168945
True : [array([0, 1, 0])] Pred : 0.5786470174789429
True : [array([1, 0, 0])] Pred : 0.22484241425991058
True : [array([0, 1, 0])] Pred : 0.5287222862243652
True : [array([1, 0, 0])] Pred : 0.24050553143024445
True : [array([0, 1, 0])] Pred : 0.6522566080093384
True : [array([0, 0, 1])] Pred : 0.6013392806053162
True : [array([1, 0, 0])] Pred : 0.4917432367801666
True : [array([0, 1, 0])] Pred : 0.6785917282104492
True : [array([1, 0, 0])] Pred : 0.1775808185338974
True : [array([0, 0, 1])] Pred : 0.6341295838356018
True : [array([0, 1, 0])] Pred : 0.6694614887237549
True : [array([1, 0, 0])] Pred : 0.3929294943809509
True : [array([0, 0, 1])] Pred : 0.5815251469612122
True : [array([0, 1, 0])] Pred : 0.6648415923118591
True : [array([1, 0, 0])] Pred : 0.24275396764278412
True : [array([0, 0, 1])] Pred : 0.5990720987319946
True : [array([0, 0, 1])] Pred : 0.5854787230491638
True : [a

  0%|                                                                                       | 0/100000 [00:00<?, ?it/s]

True : [array([0, 1, 0])] Pred : 0.6875989437103271
True : [array([0, 1, 0])] Pred : 0.6781052350997925
True : [array([0, 1, 0])] Pred : 0.6531922221183777
True : [array([0, 0, 1])] Pred : 0.6004617810249329
True : [array([0, 1, 0])] Pred : 0.6805343627929688
True : [array([0, 1, 0])] Pred : 0.6829662322998047
True : [array([0, 1, 0])] Pred : 0.6763188242912292
True : [array([0, 0, 1])] Pred : 0.6096765995025635
True : [array([1, 0, 0])] Pred : 0.36232563853263855
True : [array([0, 1, 0])] Pred : 0.6822564601898193
True : [array([0, 1, 0])] Pred : 0.627036988735199
True : [array([1, 0, 0])] Pred : 0.27827394008636475
True : [array([0, 1, 0])] Pred : 0.4817928373813629
True : [array([0, 1, 0])] Pred : 0.6399457454681396
True : [array([1, 0, 0])] Pred : 0.2765539884567261
True : [array([0, 0, 1])] Pred : 0.6395446062088013
True : [array([0, 1, 0])] Pred : 0.4772101938724518
True : [array([1, 0, 0])] Pred : 0.2986043393611908
True : [array([1, 0, 0])] Pred : 0.45233118534088135
True : [ar

TypeError: only size-1 arrays can be converted to Python scalars